## Text Classification Model Analysis

In this notebook, we analyze the model to understand the effect of graph convolution on the text classification task.

* Wha kind of graph structure is effective?
  * Node representation
  * Edge definition
* What kind of connection is effective?
  * dependency type etc.


In [1]:
import os
import sys
import numpy as np
import pandas as pd
import spacy
import matplotlib as plt
from sklearn.metrics import classification_report

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
def set_path():
    root = os.path.join(os.path.realpath("."), "../../")
    if root not in sys.path:
        sys.path.append(root)
    return root

ROOT_DIR = set_path()

## Graph Structure Experiments


Node representation

* Word embedding only
* LSTM before Graph Convolution
* LSTM after Graph Convolution

LSTM: bidirectional/LSTM

Edge kind

* Dependency
* Similarity


In [4]:
from sklearn.metrics import classification_report
from gcn.data.multi_nli_dataset import MultiNLIDataset
from gcn.classification.trainer import Trainer
from gcn.classification.graph_based_classifier import GraphBasedClassifier


def experiment(graph_type, lstm, bidirectional=False):
    dataset = MultiNLIDataset(ROOT_DIR)
    trainer = Trainer(ROOT_DIR, log_dir="classifier")
    if graph_type == "dependency":
        trainer.build_dependency_graph_trainer()        
    else:
        trainer.build_similarity_graph_trainer()

    sequence_length = 25
    vocab_size = len(trainer.preprocessor.vocabulary.get())

    def preprocessor(x):
        _x = trainer.preprocess(x, sequence_length)
        values = (_x["text"], _x["graph"])
        return values

    model = GraphBasedClassifier(vocab_size, sequence_length,
                                                        lstm=lstm, bidirectional=bidirectional)
    model.build(trainer.num_classes, preprocessor)
    metrics = trainer.train(model.model, epochs=25, verbose=0)

    result = pd.DataFrame.from_dict(metrics.history)
    result.plot.line(secondary_y=["loss", "val_loss"])

    test_data = dataset.test_data()
    y_pred = model.predict(test_data["text"])

    print(classification_report(test_data["label"], y_pred,
                                target_names=dataset.labels()))
    
    return model

### Word Embedding Only

In [ ]:
model_d = experiment("dependency", lstm=None)

Load existing preprocessor preprocessor.pkl.


In [ ]:
model_s = experiment("similarity", lstm=None)

### +LSTM

In [ ]:
model_d_b = experiment("dependency", lstm="before")

In [ ]:
model_d_a =experiment("dependency", lstm="after")

In [ ]:
model_s_b =experiment("similarity", lstm="before")

In [ ]:
model_s_a =experiment("similarity", lstm="after")

### Bidirectional

In [ ]:
model_d_b_b = experiment("dependency", lstm="before", bidirectional=True)

In [ ]:
model_s_b_b = experiment("similarity", lstm="before", bidirectional=True)